# **Chapter 19: HTTP Requests & REST APIs**

---

## **Learning Objectives**

By the end of this chapter, you will be able to:

- Compare and select between `http` and `dio` packages for different use cases
- Implement CRUD operations (GET, POST, PUT, DELETE) with proper error handling
- Serialize and deserialize JSON data using manual parsing and code generation
- Set up and use `json_serializable` with `build_runner` for type-safe API models
- Implement immutable data classes using `Freezed` for robust state management
- Design resilient network layers with retry logic, timeouts, and interceptors
- Handle network errors gracefully with proper exception hierarchies
- Structure API clients following repository pattern and clean architecture principles

---

## **Prerequisites**

- Completed Chapter 6: Asynchronous Programming (Futures, async/await)
- Completed Chapter 5: Object-Oriented Dart (Classes, Generics, Enums)
- Understanding of REST API concepts (endpoints, HTTP methods, status codes)
- Basic knowledge of JSON data structure
- Flutter SDK installed with a working emulator or physical device

---

## **19.1 Choosing Your HTTP Client: `http` vs `dio`**

Dart's ecosystem offers two primary HTTP clients: the official `http` package and the community-favorite `dio` package. Understanding their differences is crucial for architectural decisions.

### **The `http` Package**

The `http` package is the official Dart package maintained by the Dart team. It's lightweight, simple, and sufficient for basic HTTP operations.

```dart
// Add to pubspec.yaml:
// dependencies:
//   http: ^1.1.0

import 'package:http/http.dart' as http;
import 'dart:convert';

class HttpExample {
  // Base URL for the API
  static const String baseUrl = 'https://api.example.com';
  
  // GET request example
  Future<void> fetchUser(String userId) async {
    // Construct the full URL
    final url = Uri.parse('$baseUrl/users/$userId');
    
    try {
      // http.get returns a Future<Response>
      // await pauses execution until the response is received
      final response = await http.get(
        url,
        headers: {
          'Content-Type': 'application/json',
          'Accept': 'application/json',
        },
      );
      
      // Check the status code
      // 200 indicates success (OK)
      if (response.statusCode == 200) {
        // response.body is a String containing JSON
        // jsonDecode converts JSON string to Dart Map
        final data = jsonDecode(response.body);
        print('User data: $data');
      } else {
        // Handle non-200 status codes
        throw Exception('Failed to load user: ${response.statusCode}');
      }
    } catch (e) {
      // Handle network errors (no connection, timeout, etc.)
      print('Error fetching user: $e');
      rethrow; // Propagate error to caller
    }
  }
  
  // POST request example
  Future<void> createUser(String name, String email) async {
    final url = Uri.parse('$baseUrl/users');
    
    try {
      final response = await http.post(
        url,
        headers: {
          'Content-Type': 'application/json',
        },
        // jsonEncode converts Dart Map to JSON string
        body: jsonEncode({
          'name': name,
          'email': email,
        }),
      );
      
      if (response.statusCode == 201) { // 201 Created
        final data = jsonDecode(response.body);
        print('Created user: $data');
      } else {
        throw Exception('Failed to create user: ${response.statusCode}');
      }
    } catch (e) {
      print('Error creating user: $e');
      rethrow;
    }
  }
}
```

**Explanation:**

- **`import 'package:http/http.dart' as http`**: Imports the http package with a prefix to avoid naming conflicts.
- **`Uri.parse()`**: Converts a string URL to a Uri object, which handles encoding and validation.
- **`await http.get()`**: Makes an asynchronous GET request. The `await` keyword pauses execution until the Future completes.
- **`headers`**: HTTP headers specify metadata. `Content-Type: application/json` tells the server we're sending/expecting JSON.
- **`response.statusCode`**: HTTP status codes indicate success (200-299), client errors (400-499), or server errors (500-599).
- **`jsonDecode()`**: Converts JSON string (from `response.body`) into a Dart Map or List.
- **`jsonEncode()`**: Converts Dart objects to JSON string for the request body.
- **`try-catch`**: Essential for handling network exceptions (SocketException, TimeoutException).

### **The `dio` Package**

`dio` is a powerful HTTP client for Dart, which supports Interceptors, Global configuration, FormData, Request Cancellation, File downloading/uploading, etc.

```dart
// Add to pubspec.yaml:
// dependencies:
//   dio: ^5.3.0

import 'package:dio/dio.dart';

class DioExample {
  // Dio instance with base configuration
  late final Dio _dio;
  
  // Constructor initializes Dio with configuration
  DioExample() {
    _dio = Dio(
      BaseOptions(
        // Base URL for all requests
        baseUrl: 'https://api.example.com',
        // Default headers for all requests
        headers: {
          'Content-Type': 'application/json',
          'Accept': 'application/json',
        },
        // Connection timeout duration
        connectTimeout: Duration(seconds: 5),
        // Receive timeout duration
        receiveTimeout: Duration(seconds: 3),
      ),
    );
    
    // Add interceptors for logging and error handling
    _dio.interceptors.add(
      InterceptorsWrapper(
        onRequest: (options, handler) {
          // Modify request before sending
          print('Request: ${options.method} ${options.path}');
          // Add auth token if available
          options.headers['Authorization'] = 'Bearer token123';
          return handler.next(options); // Continue
        },
        onResponse: (response, handler) {
          // Handle response before returning to caller
          print('Response: ${response.statusCode}');
          return handler.next(response);
        },
        onError: (DioException e, handler) {
          // Handle errors globally
          print('Error: ${e.message}');
          return handler.next(e); // Propagate error
        },
      ),
    );
  }
  
  // GET request with query parameters
  Future<List<dynamic>> fetchUsers({int? page, int? limit}) async {
    try {
      // Response object contains rich information
      final response = await _dio.get(
        '/users',
        queryParameters: {
          if (page != null) 'page': page,
          if (limit != null) 'limit': limit,
        },
      );
      
      // Dio automatically parses JSON if Content-Type is application/json
      // response.data contains the parsed body (Map or List)
      return response.data as List<dynamic>;
    } on DioException catch (e) {
      // Dio-specific exception handling
      if (e.response != null) {
        // Server returned an error response
        print('Server error: ${e.response?.statusCode}');
        print('Error data: ${e.response?.data}');
      } else {
        // Network error (no connection, timeout, etc.)
        print('Network error: ${e.message}');
      }
      rethrow;
    }
  }
  
  // POST request with type safety
  Future<Map<String, dynamic>> createUser(
    String name,
    String email,
  ) async {
    try {
      final response = await _dio.post(
        '/users',
        data: {
          'name': name,
          'email': email,
        },
      );
      
      return response.data as Map<String, dynamic>;
    } on DioException catch (e) {
      _handleError(e);
      rethrow;
    }
  }
  
  // Error handling helper
  void _handleError(DioException e) {
    switch (e.type) {
      case DioExceptionType.connectionTimeout:
        print('Connection timed out');
        break;
      case DioExceptionType.receiveTimeout:
        print('Receive timeout');
        break;
      case DioExceptionType.badResponse:
        print('Server error: ${e.response?.statusCode}');
        break;
      case DioExceptionType.connectionError:
        print('No internet connection');
        break;
      default:
        print('Unknown error: ${e.message}');
    }
  }
}
```

**Explanation:**

- **`Dio()`**: Creates a Dio instance. Can be configured globally using `BaseOptions`.
- **`BaseOptions`**: Centralized configuration for base URL, headers, timeouts, and other defaults.
- **`Interceptors`**: Middleware that intercepts requests, responses, and errors. Useful for logging, authentication, and global error handling.
- **`onRequest`**: Callback fired before request is sent. Can modify headers, add tokens, or log requests.
- **`queryParameters`**: Automatically encodes query parameters into the URL (e.g., `?page=1&limit=10`).
- **`DioException`**: Rich error object containing type, message, request, and response information.
- **Automatic JSON parsing**: Dio automatically converts JSON responses to Dart objects if the Content-Type header is `application/json`.

### **Comparison and Selection Guide**

```dart
// Decision helper class
class HttpClientSelection {
  // Use http package when:
  // - Simple CRUD operations
  // - Minimal dependencies (official package)
  // - Learning purposes
  // - Small projects with basic needs
  
  // Use dio package when:
  // - Complex request/response handling
  // - Need interceptors for auth/logging
  // - File uploads/downloads with progress
  // - Request cancellation needed
  // - Global error handling required
  // - REST API with many endpoints
  
  // Industry Standard Recommendation:
  // For production apps, use Dio with Repository Pattern
  // For simple prototyping or learning, use http
}
```

**Industry Standard**: For production applications, `dio` is preferred due to its robust error handling, interceptor support, and extensive features. The `http` package is suitable for simple use cases or when minimizing dependencies is critical.

---

## **19.2 REST API Operations: CRUD Implementation**

This section implements a complete CRUD (Create, Read, Update, Delete) API client following industry best practices.

### **API Client Architecture**

```dart
import 'package:dio/dio.dart';

// Abstract class defining the contract
abstract class ApiClient {
  Future<Response> get(String path, {Map<String, dynamic>? queryParams});
  Future<Response> post(String path, {dynamic data});
  Future<Response> put(String path, {dynamic data});
  Future<Response> delete(String path);
}

// Concrete implementation using Dio
class DioApiClient implements ApiClient {
  final Dio _dio;
  
  // Dependency injection allows testing with mock Dio
  DioApiClient(this._dio) {
    _setupInterceptors();
  }
  
  void _setupInterceptors() {
    _dio.interceptors.add(
      InterceptorsWrapper(
        onRequest: (options, handler) {
          // Add timestamp to prevent caching
          options.queryParameters['_timestamp'] = 
              DateTime.now().millisecondsSinceEpoch;
          return handler.next(options);
        },
        onError: (error, handler) async {
          // Global error handling
          if (error.response?.statusCode == 401) {
            // Handle token refresh or logout
            print('Unauthorized - redirect to login');
          }
          return handler.next(error);
        },
      ),
    );
  }
  
  @override
  Future<Response> get(
    String path, {
    Map<String, dynamic>? queryParams,
  }) async {
    try {
      return await _dio.get(
        path,
        queryParameters: queryParams,
      );
    } on DioException catch (e) {
      throw _handleError(e);
    }
  }
  
  @override
  Future<Response> post(String path, {dynamic data}) async {
    try {
      return await _dio.post(path, data: data);
    } on DioException catch (e) {
      throw _handleError(e);
    }
  }
  
  @override
  Future<Response> put(String path, {dynamic data}) async {
    try {
      return await _dio.put(path, data: data);
    } on DioException catch (e) {
      throw _handleError(e);
    }
  }
  
  @override
  Future<Response> delete(String path) async {
    try {
      return await _dio.delete(path);
    } on DioException catch (e) {
      throw _handleError(e);
    }
  }
  
  // Convert DioException to domain-specific exceptions
  Exception _handleError(DioException e) {
    switch (e.type) {
      case DioExceptionType.connectionTimeout:
      case DioExceptionType.sendTimeout:
      case DioExceptionType.receiveTimeout:
        return TimeoutException();
      case DioExceptionType.badResponse:
        return ApiException(
          statusCode: e.response?.statusCode ?? 0,
          message: e.response?.data?['message'] ?? 'Unknown error',
        );
      case DioExceptionType.connectionError:
        return NetworkException();
      default:
        return UnknownException(e.message ?? 'Unknown error');
    }
  }
}

// Custom exception classes for domain-specific error handling
class TimeoutException implements Exception {
  final String message = 'Request timed out. Please try again.';
}

class NetworkException implements Exception {
  final String message = 'No internet connection. Check your network.';
}

class ApiException implements Exception {
  final int statusCode;
  final String message;
  
  ApiException({required this.statusCode, required this.message});
  
  @override
  String toString() => 'ApiException($statusCode): $message';
}

class UnknownException implements Exception {
  final String message;
  UnknownException(this.message);
}
```

**Explanation:**

- **`abstract class ApiClient`**: Defines a contract that concrete implementations must follow. Enables swapping implementations (e.g., for testing).
- **Dependency Injection**: Accepting `Dio` via constructor allows injecting mock instances for unit testing.
- **Interceptor Logic**: Adds timestamps to prevent caching, handles 401 unauthorized globally.
- **Error Translation**: Converts Dio-specific errors into domain-specific exceptions that the UI layer can handle appropriately.
- **Custom Exceptions**: Creating specific exception types allows the UI to show appropriate error messages (e.g., "Check your connection" vs "Server error").

### **Complete CRUD Implementation**

```dart
// User model (simplified - will be replaced with generated model later)
class User {
  final String id;
  final String name;
  final String email;
  final DateTime createdAt;
  
  User({
    required this.id,
    required this.name,
    required this.email,
    required this.createdAt,
  });
  
  // Manual JSON deserialization (fromJson)
  factory User.fromJson(Map<String, dynamic> json) {
    return User(
      id: json['id'] as String,
      name: json['name'] as String,
      email: json['email'] as String,
      createdAt: DateTime.parse(json['created_at'] as String),
    );
  }
  
  // Manual JSON serialization (toJson)
  Map<String, dynamic> toJson() {
    return {
      'id': id,
      'name': name,
      'email': email,
      'created_at': createdAt.toIso8601String(),
    };
  }
}

// Repository following Clean Architecture principles
class UserRepository {
  final ApiClient _apiClient;
  final String _basePath = '/users';
  
  UserRepository(this._apiClient);
  
  // READ (GET) - Fetch single user
  Future<User> getUser(String id) async {
    final response = await _apiClient.get('$_basePath/$id');
    
    // Validate response data
    if (response.data == null) {
      throw ApiException(statusCode: 0, message: 'Empty response');
    }
    
    return User.fromJson(response.data as Map<String, dynamic>);
  }
  
  // READ (GET) - Fetch list with pagination
  Future<List<User>> getUsers({
    int page = 1,
    int limit = 20,
  }) async {
    final response = await _apiClient.get(
      _basePath,
      queryParams: {
        'page': page,
        'limit': limit,
      },
    );
    
    // Handle list response
    final List<dynamic> data = response.data as List<dynamic>;
    return data.map((json) => User.fromJson(json as Map<String, dynamic>)).toList();
  }
  
  // CREATE (POST)
  Future<User> createUser({
    required String name,
    required String email,
  }) async {
    final response = await _apiClient.post(
      _basePath,
      data: {
        'name': name,
        'email': email,
      },
    );
    
    return User.fromJson(response.data as Map<String, dynamic>);
  }
  
  // UPDATE (PUT) - Full update
  Future<User> updateUser(User user) async {
    final response = await _apiClient.put(
      '$_basePath/${user.id}',
      data: user.toJson(),
    );
    
    return User.fromJson(response.data as Map<String, dynamic>);
  }
  
  // UPDATE (PATCH) - Partial update
  Future<User> patchUser(
    String id, {
    String? name,
    String? email,
  }) async {
    // Build payload with only provided fields
    final Map<String, dynamic> data = {};
    if (name != null) data['name'] = name;
    if (email != null) data['email'] = email;
    
    final response = await _apiClient.put(
      '$_basePath/$id',
      data: data,
    );
    
    return User.fromJson(response.data as Map<String, dynamic>);
  }
  
  // DELETE
  Future<void> deleteUser(String id) async {
    await _apiClient.delete('$_basePath/$id');
    // No content expected for DELETE (204 No Content)
  }
}
```

**Explanation:**

- **Repository Pattern**: `UserRepository` abstracts data source details from the rest of the app. The UI layer works with repositories, not API clients directly.
- **fromJson/toJson**: Standard methods for converting between Dart objects and JSON Maps.
- **Pagination**: `page` and `limit` parameters implement offset-based pagination.
- **PUT vs PATCH**: PUT replaces the entire resource, while PATCH updates only specified fields.
- **Type Safety**: Casting `response.data` to specific types ensures compile-time safety.
- **Null Safety**: Checking for null responses prevents runtime crashes.

---

## **19.3 JSON Serialization with `json_serializable`**

Manual JSON parsing is error-prone and tedious. The `json_serializable` package automates this process using code generation.

### **Setup and Configuration**

```dart
// pubspec.yaml dependencies:
// dependencies:
//   json_annotation: ^4.8.1
// 
// dev_dependencies:
//   build_runner: ^2.4.6
//   json_serializable: ^6.7.1

// Run code generation:
// dart run build_runner build        (one-time)
// dart run build_runner watch        (continuous during development)
// dart run build_runner build --delete-conflicting-outputs    (force rebuild)
```

**Explanation:**

- **`json_annotation`**: Runtime package containing annotations used to mark classes for generation.
- **`json_serializable`**: Dev dependency that generates serialization code.
- **`build_runner`**: Tool that runs code generators.
- **`build`**: Generates code once.
- **`watch`**: Watches for file changes and regenerates automatically.
- **`--delete-conflicting-outputs`**: Resolves conflicts when generated files exist.

### **Annotated Model Classes**

```dart
import 'package:json_annotation/json_annotation.dart';

// This line tells the generator to create a file named user.g.dart
// The 'part' directive includes the generated file
part 'user.g.dart';

// @JsonSerializable() marks this class for code generation
@JsonSerializable()
class UserModel {
  // Field mapping: if JSON key differs from Dart field name
  @JsonKey(name: 'id') // Explicit mapping (optional if names match)
  final String id;
  
  final String name;
  final String email;
  
  // Custom mapping for nested objects or transformations
  @JsonKey(name: 'created_at') // Maps JSON snake_case to Dart camelCase
  final DateTime createdAt;
  
  // Nullable field with default value
  @JsonKey(defaultValue: 'active')
  final String status;
  
  // Nullable complex type
  final AddressModel? address;
  
  // List of complex objects
  final List<String>? tags;

  UserModel({
    required this.id,
    required this.name,
    required this.email,
    required this.createdAt,
    this.status = 'active',
    this.address,
    this.tags,
  });

  // Generated factory constructor
  // _$UserModelFromJson is generated in user.g.dart
  factory UserModel.fromJson(Map<String, dynamic> json) => 
      _$UserModelFromJson(json);

  // Generated toJson method
  Map<String, dynamic> toJson() => _$UserModelToJson(this);
}

@JsonSerializable()
class AddressModel {
  final String street;
  final String city;
  
  @JsonKey(name: 'zip_code')
  final String zipCode;

  AddressModel({
    required this.street,
    required this.city,
    required this.zipCode,
  });

  factory AddressModel.fromJson(Map<String, dynamic> json) => 
      _$AddressModelFromJson(json);

  Map<String, dynamic> toJson() => _$AddressModelToJson(this);
}
```

**Explanation:**

- **`part 'user.g.dart'`**: Includes the generated file. The `.g.dart` extension is convention for generated files.
- **`@JsonSerializable()`**: Annotation that triggers code generation for this class.
- **`@JsonKey()`**: Customizes field mapping:
  - `name`: Maps JSON keys to different Dart field names (e.g., snake_case to camelCase).
  - `defaultValue`: Provides default if JSON field is missing or null.
  - `ignore`: Excludes field from serialization.
  - `fromJson`/`toJson`: Custom conversion functions.
- **`_$UserModelFromJson`**: Generated function that handles type checking, null safety, and nested object deserialization.
- **Nullable types**: `?` indicates optional fields that may be null in JSON.

### **Advanced Serialization Patterns**

```dart
import 'package:json_annotation/json_annotation.dart';

part 'product.g.dart';

// Enum with JSON value mapping
enum ProductStatus {
  @JsonValue('available') // Maps to string "available" in JSON
  available,
  @JsonValue('out_of_stock')
  outOfStock,
  @JsonValue('discontinued')
  discontinued,
}

@JsonSerializable()
class ProductModel {
  final String id;
  final String name;
  final double price;
  final ProductStatus status;
  
  // Custom DateTime format
  @JsonKey(fromJson: _dateTimeFromJson, toJson: _dateTimeToJson)
  final DateTime expiryDate;
  
  // Computed property (not serialized)
  @JsonKey(ignore: true)
  bool get isAvailable => status == ProductStatus.available;

  ProductModel({
    required this.id,
    required this.name,
    required this.price,
    required this.status,
    required this.expiryDate,
  });

  factory ProductModel.fromJson(Map<String, dynamic> json) => 
      _$ProductModelFromJson(json);

  Map<String, dynamic> toJson() => _$ProductModelToJson(this);

  // Custom JSON converter for DateTime
  static DateTime _dateTimeFromJson(String date) => DateTime.parse(date);
  static String _dateTimeToJson(DateTime date) => date.toIso8601String();
}

// Generic response wrapper for API consistency
@JsonSerializable(genericArgumentFactories: true)
class ApiResponse<T> {
  final bool success;
  final String? message;
  final T? data;
  
  // Generic deserialization requires type helper
  @JsonKey(fromJson: _dataFromJson, toJson: _dataToJson)
  final T dataHelper;

  ApiResponse({
    required this.success,
    this.message,
    required this.dataHelper,
  }) : data = dataHelper;

  factory ApiResponse.fromJson(
    Map<String, dynamic> json,
    T Function(Object? json) fromJsonT,
  ) => 
      _$ApiResponseFromJson(json, fromJsonT);

  Map<String, dynamic> toJson(Object? Function(T value) toJsonT) => 
      _$ApiResponseToJson(this, toJsonT);

  static T _dataFromJson<T>(Object? json, T Function(Object?) fromJson) => 
      fromJson(json);
  
  static Object? _dataToJson<T>(T data, Object? Function(T) toJson) => 
      toJson(data);
}

// Usage with specific types
@JsonSerializable()
class ProductListResponse {
  final List<ProductModel> products;
  final int total;

  ProductListResponse({required this.products, required this.total});

  factory ProductListResponse.fromJson(Map<String, dynamic> json) => 
      _$ProductListResponseFromJson(json);
}
```

**Explanation:**

- **`@JsonValue`**: Maps enum values to specific JSON strings.
- **Custom converters**: Static functions handle non-standard types (e.g., custom date formats).
- **Generic responses**: `ApiResponse<T>` wraps API responses with metadata (success, message) and typed data.
- **`genericArgumentFactories: true`**: Enables code generation for generic classes.
- **Computed properties**: `@JsonKey(ignore: true)` excludes getters from serialization.

---

## **19.4 Immutable Data Classes with `Freezed`**

`Freezed` is a powerful code generator that creates immutable data classes with copy methods, JSON serialization, and union types.

### **Setup and Basic Usage**

```dart
// pubspec.yaml:
// dependencies:
//   freezed_annotation: ^2.4.1
//   json_annotation: ^4.8.1
//
// dev_dependencies:
//   build_runner: ^2.4.6
//   freezed: ^2.4.2
//   json_serializable: ^6.7.1

import 'package:freezed_annotation/freezed_annotation.dart';

part 'user.freezed.dart';
part 'user.g.dart';

// @freezed annotation triggers generation of immutable class
@freezed
class User with _$User {
  // Private constructor for custom methods
  const User._();

  // Factory constructor with named parameters
  const factory User({
    required String id,
    required String name,
    required String email,
    @Default('active') String status, // Default value
    DateTime? createdAt, // Nullable
  }) = _User;

  // JSON serialization factory
  factory User.fromJson(Map<String, dynamic> json) => _$UserFromJson(json);

  // Custom computed property
  bool get isActive => status == 'active';
}

// Usage example
void main() {
  // Create instance
  final user = User(
    id: '1',
    name: 'John',
    email: 'john@example.com',
  );
  
  // Copy with modification (immutable update)
  final updatedUser = user.copyWith(name: 'Johnny');
  // user remains unchanged, updatedUser is a new instance
  
  // Equality comparison (deep equality)
  print(user == updatedUser); // false
  
  // JSON serialization
  final json = user.toJson();
  final fromJson = User.fromJson(json);
}
```

**Explanation:**

- **`@freezed`**: Generates immutable classes with value equality, copy methods, and toString.
- **`with _$User`**: Mixin that provides generated methods (`copyWith`, `toJson`, etc.).
- **`const factory`**: Creates a const-capable constructor. The `= _User` part assigns to a private generated class.
- **`@Default(value)`**: Provides default values for optional parameters.
- **`copyWith`**: Generated method that creates a copy with modified fields. Since the class is immutable, this is the only way to "modify" instances.
- **Value equality**: Freezed classes override `==` and `hashCode` based on field values, not identity.

### **Union Types (Sealed Classes)**

Freezed excels at creating union types (sealed classes) for representing different states.

```dart
import 'package:freezed_annotation/freezed_annotation.dart';

part 'api_state.freezed.dart';

// Union type representing API states
@freezed
class ApiState<T> with _$ApiState<T> {
  // Initial state
  const factory ApiState.initial() = ApiInitial<T>;
  
  // Loading state
  const factory ApiState.loading() = ApiLoading<T>;
  
  // Success state with data
  const factory ApiState.data(T value) = ApiData<T>;
  
  // Error state with exception
  const factory ApiState.error(String message) = ApiError<T>;
}

// Usage in BLoC/State Management
class UserCubit extends Cubit<ApiState<User>> {
  UserCubit() : super(const ApiState.initial());
  
  Future<void> fetchUser(String id) async {
    emit(const ApiState.loading());
    
    try {
      final user = await _repository.getUser(id);
      emit(ApiState.data(user));
    } catch (e) {
      emit(ApiState.error(e.toString()));
    }
  }
}

// UI handling with pattern matching
Widget buildUserWidget(ApiState<User> state) {
  return state.when(
    initial: () => const SizedBox.shrink(),
    loading: () => const CircularProgressIndicator(),
    data: (user) => UserProfile(user: user),
    error: (message) => ErrorWidget(message: message),
  );
}
```

**Explanation:**

- **Union types**: Multiple factories represent different states of the same concept.
- **Pattern matching**: The `.when()` method requires handling all cases (exhaustive matching), preventing missed states.
- **Type safety**: Each state carries its specific data (e.g., `ApiData` contains `T`, `ApiError` contains `String`).
- **Other methods**: `.map()` for transforming values, `.maybeWhen()` for handling specific cases with fallback.

---

## **19.5 Error Handling and Retry Logic**

Production apps require robust error handling and automatic retry mechanisms for transient failures.

### **Retry Interceptor**

```dart
import 'package:dio/dio.dart';
import 'dart:math';

class RetryInterceptor extends Interceptor {
  final int maxRetries;
  final Duration retryDelay;
  final Set<int> retryStatusCodes;
  final Random _random = Random();

  RetryInterceptor({
    this.maxRetries = 3,
    this.retryDelay = const Duration(seconds: 1),
    this.retryStatusCodes = const {408, 429, 500, 502, 503, 504},
  });

  @override
  void onError(DioException err, ErrorInterceptorHandler handler) async {
    // Check if we should retry
    if (_shouldRetry(err) && err.requestOptions.extra['retryCount'] != maxRetries) {
      final retryCount = (err.requestOptions.extra['retryCount'] as int?) ?? 0;
      
      if (retryCount < maxRetries) {
        // Exponential backoff with jitter
        final delay = _calculateDelay(retryCount);
        print('Retrying request (${retryCount + 1}/$maxRetries) after ${delay.inMilliseconds}ms');
        
        await Future.delayed(delay);
        
        // Update retry count in request options
        err.requestOptions.extra['retryCount'] = retryCount + 1;
        
        // Retry the request
        try {
          final response = await _dio.fetch(err.requestOptions);
          handler.resolve(response);
          return;
        } catch (e) {
          // If retry fails, continue to error handler
        }
      }
    }
    
    // If we shouldn't retry or exhausted retries, pass error along
    handler.next(err);
  }

  bool _shouldRetry(DioException error) {
    // Retry on timeout errors
    if (error.type == DioExceptionType.connectionTimeout ||
        error.type == DioExceptionType.sendTimeout ||
        error.type == DioExceptionType.receiveTimeout) {
      return true;
    }
    
    // Retry on specific status codes
    if (error.response != null && 
        retryStatusCodes.contains(error.response?.statusCode)) {
      return true;
    }
    
    // Retry on connection errors
    if (error.type == DioExceptionType.connectionError) {
      return true;
    }
    
    return false;
  }

  Duration _calculateDelay(int retryCount) {
    // Exponential backoff: 2^retryCount * baseDelay
    // Plus jitter: random value between 0 and 100ms to prevent thundering herd
    final exponentialDelay = retryDelay * pow(2, retryCount);
    final jitter = Duration(milliseconds: _random.nextInt(100));
    return exponentialDelay + jitter;
  }
  
  late final Dio _dio; // Set when added to Dio instance
}

// Extension to set the Dio instance
extension RetryInterceptorExtension on RetryInterceptor {
  void attachTo(Dio dio) {
    (this as dynamic)._dio = dio;
    dio.interceptors.add(this);
  }
}
```

**Explanation:**

- **Interceptor**: Extends `Interceptor` to hook into the request lifecycle.
- **Exponential Backoff**: Delay increases exponentially (1s, 2s, 4s, 8s) to avoid overwhelming the server.
- **Jitter**: Random component prevents synchronized retries from multiple clients (thundering herd problem).
- **Retry Count**: Stored in request `extra` map to track across attempts.
- **Status Codes**: Retries on 408 (Timeout), 429 (Too Many Requests), and 5xx server errors.

### **Comprehensive Error Handling Strategy**

```dart
// Result type for functional error handling (Either pattern)
abstract class Result<T> {
  const Result();
  
  // Factory constructors for success and failure
  const factory Result.success(T data) = Success<T>;
  const factory Result.failure(ApiError error) = Failure<T>;
  
  // Pattern matching methods
  R when<R>({
    required R Function(T data) success,
    required R Function(ApiError error) failure,
  });
}

class Success<T> extends Result<T> {
  final T data;
  const Success(this.data);
  
  @override
  R when<R>({
    required R Function(T data) success,
    required R Function(ApiError error) failure,
  }) => success(data);
}

class Failure<T> extends Result<T> {
  final ApiError error;
  const Failure(this.error);
  
  @override
  R when<R>({
    required R Function(T data) success,
    required R Function(ApiError error) failure,
  }) => failure(error);
}

// Domain-specific error types
class ApiError {
  final ErrorType type;
  final String message;
  final int? statusCode;
  final dynamic originalError;

  ApiError({
    required this.type,
    required this.message,
    this.statusCode,
    this.originalError,
  });

  factory ApiError.fromException(dynamic error) {
    if (error is DioException) {
      return _handleDioError(error);
    }
    return ApiError(
      type: ErrorType.unknown,
      message: 'An unexpected error occurred',
      originalError: error,
    );
  }

  static ApiError _handleDioError(DioException error) {
    switch (error.type) {
      case DioExceptionType.connectionTimeout:
      case DioExceptionType.sendTimeout:
      case DioExceptionType.receiveTimeout:
        return ApiError(
          type: ErrorType.timeout,
          message: 'Connection timed out. Please check your internet.',
          statusCode: 0,
          originalError: error,
        );
      case DioExceptionType.badResponse:
        return _handleHttpError(error.response!);
      case DioExceptionType.connectionError:
        return ApiError(
          type: ErrorType.network,
          message: 'No internet connection.',
          originalError: error,
        );
      default:
        return ApiError(
          type: ErrorType.unknown,
          message: error.message ?? 'Unknown error',
          originalError: error,
        );
    }
  }

  static ApiError _handleHttpError(Response response) {
    final statusCode = response.statusCode ?? 0;
    String message = 'Server error';
    
    // Parse error message from response body if available
    if (response.data is Map) {
      message = response.data['message'] ?? 
                response.data['error'] ?? 
                'Request failed';
    }
    
    return ApiError(
      type: _mapStatusCodeToErrorType(statusCode),
      message: message,
      statusCode: statusCode,
      originalError: response,
    );
  }

  static ErrorType _mapStatusCodeToErrorType(int code) {
    if (code == 400) return ErrorType.badRequest;
    if (code == 401) return ErrorType.unauthorized;
    if (code == 403) return ErrorType.forbidden;
    if (code == 404) return ErrorType.notFound;
    if (code == 409) return ErrorType.conflict;
    if (code == 422) return ErrorType.validation;
    if (code >= 500) return ErrorType.server;
    return ErrorType.unknown;
  }
}

enum ErrorType {
  network,
  timeout,
  badRequest,
  unauthorized,
  forbidden,
  notFound,
  conflict,
  validation,
  server,
  unknown,
}

// Repository with Result type
class SafeUserRepository {
  final ApiClient _client;

  SafeUserRepository(this._client);

  Future<Result<User>> getUser(String id) async {
    try {
      final response = await _client.get('/users/$id');
      final user = User.fromJson(response.data);
      return Result.success(user);
    } catch (e) {
      return Result.failure(ApiError.fromException(e));
    }
  }

  Future<Result<List<User>>> getUsers() async {
    try {
      final response = await _client.get('/users');
      final users = (response.data as List)
          .map((json) => User.fromJson(json))
          .toList();
      return Result.success(users);
    } catch (e) {
      return Result.failure(ApiError.fromException(e));
    }
  }
}

// UI Layer handling
class UserController {
  final SafeUserRepository _repository;
  String? _errorMessage;

  UserController(this._repository);

  Future<void> loadUser(String id) async {
    final result = await _repository.getUser(id);
    
    result.when(
      success: (user) {
        // Update UI with user data
        print('Loaded user: ${user.name}');
        _errorMessage = null;
      },
      failure: (error) {
        // Handle specific error types
        _errorMessage = _getUserFriendlyMessage(error);
        print('Error: $_errorMessage');
      },
    );
  }

  String _getUserFriendlyMessage(ApiError error) {
    switch (error.type) {
      case ErrorType.network:
        return 'Please check your internet connection and try again.';
      case ErrorType.timeout:
        return 'The request took too long. Please try again.';
      case ErrorType.unauthorized:
        return 'Your session has expired. Please log in again.';
      case ErrorType.notFound:
        return 'The requested resource was not found.';
      case ErrorType.validation:
        return 'Please check your input and try again.';
      case ErrorType.server:
        return 'Our server is having issues. Please try again later.';
      default:
        return 'Something went wrong. Please try again.';
    }
  }
}
```

**Explanation:**

- **Result Type**: Functional programming pattern that explicitly represents success or failure. Forces error handling at compile time.
- **Exhaustive Error Types**: Specific error types allow the UI to show appropriate messages and actions (e.g., "Login again" for 401, "Check connection" for network errors).
- **Error Mapping**: Converts technical exceptions (DioException) to user-friendly domain errors.
- **Pattern Matching**: `when()` method ensures all cases are handled, preventing unhandled errors.
- **Separation of Concerns**: Repository handles technical details, UI layer handles presentation of errors.

### **Connectivity Awareness**

```dart
import 'package:connectivity_plus/connectivity_plus.dart';

class NetworkAwareClient {
  final ApiClient _apiClient;
  final Connectivity _connectivity;

  NetworkAwareClient(this._apiClient, this._connectivity);

  Future<T> execute<T>(Future<T> Function() apiCall) async {
    // Check connectivity before making request
    final connectivityResult = await _connectivity.checkConnectivity();
    
    if (connectivityResult == ConnectivityResult.none) {
      throw ApiError(
        type: ErrorType.network,
        message: 'No internet connection available',
      );
    }
    
    // Listen for connectivity changes during request
    final connectivitySubscription = _connectivity.onConnectivityChanged.listen((result) {
      if (result == ConnectivityResult.none) {
        print('Connection lost during request');
      }
    });
    
    try {
      return await apiCall();
    } finally {
      await connectivitySubscription.cancel();
    }
  }
}
```

**Explanation:**

- **Pre-flight Check**: Verifies connectivity before attempting request to fail fast.
- **Connectivity Plus**: Package to monitor network state (WiFi, Mobile, None).
- **Streaming Updates**: Listens for connectivity changes during long-running requests.

---

## **Chapter Summary**

In this chapter, we covered enterprise-grade HTTP networking in Flutter:

### **Key Takeaways:**

1. **HTTP Clients**: 
   - Use `http` for simple use cases and minimal dependencies
   - Use `dio` for production apps requiring interceptors, global config, and advanced features

2. **Architecture Patterns**:
   - **Repository Pattern**: Abstracts data sources from UI/business logic
   - **ApiClient Abstraction**: Allows swapping implementations and testing
   - **Dependency Injection**: Enables testing and decoupling

3. **CRUD Operations**:
   - Implement type-safe methods for GET, POST, PUT, DELETE
   - Handle query parameters, request bodies, and headers appropriately
   - Distinguish between PUT (full update) and PATCH (partial update)

4. **JSON Serialization**:
   - **`json_serializable`**: Automates serialization boilerplate
   - **`@JsonKey()`**: Customizes field mapping and transformations
   - **Code Generation**: Run `build_runner` to generate `*.g.dart` files

5. **Immutable Data**:
   - **`Freezed`**: Generates immutable classes with copy methods, JSON support, and unions
   - **Union Types**: Represent API states (Loading, Error, Success) with type safety
   - **Pattern Matching**: Handle all cases exhaustively with `.when()`

6. **Error Handling**:
   - **Custom Exceptions**: Map technical errors to domain-specific types
   - **Result Type**: Force explicit error handling using Success/Failure pattern
   - **Retry Logic**: Implement exponential backoff with jitter for transient failures
   - **Connectivity Awareness**: Check network state before requests

7. **Production Considerations**:
   - **Interceptors**: Centralize logging, auth tokens, and error handling
   - **Timeouts**: Configure connection and receive timeouts
   - **Type Safety**: Cast responses to specific types immediately
   - **Null Safety**: Validate responses before parsing

### **Best Practices Checklist:**

- [ ] Always use Repository Pattern to abstract API calls
- [ ] Implement proper error handling with domain-specific exceptions
- [ ] Use code generation (json_serializable/Freezed) for models
- [ ] Add retry logic with exponential backoff for network resilience
- [ ] Validate JSON schema before parsing (use nullable types appropriately)
- [ ] Implement connectivity checks before network operations
- [ ] Use Result types or try-catch blocks to handle errors explicitly
- [ ] Configure proper timeouts (don't rely on defaults)
- [ ] Add request/response logging in debug mode only
- [ ] Handle token refresh and 401 errors globally via interceptors

---

## **Next Chapter: Chapter 20 - Authentication & Security**

Chapter 20 will explore secure authentication patterns in Flutter, including:

- OAuth 2.0 flows and JWT token management
- Secure storage of credentials using `flutter_secure_storage`
- Biometric authentication with `local_auth`
- Certificate pinning for API security
- API key management and environment configuration
- Handling token refresh and session management

You will learn to implement production-grade security features that protect user data and comply with industry standards.